<a href="https://colab.research.google.com/github/smkandel/Projects/blob/main/Final_Copy_of_Capstone_ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#read final data
df = pd.read_csv('/content/drive/MyDrive/Predictive_maintenance_datasets/final_data1.csv')
df.shape[0]

876545

In [5]:
#add machine data
df_machine = pd.read_csv('/content/drive/MyDrive/Predictive_maintenance_datasets/PdM_machines.csv')

#merge and override
df = df.merge(df_machine, on=['machineID'], how='left')
df

failure        datetime  machineID        volt      rotate    pressure  \
0           no   1/1/2015 6:00          1  176.217853  418.504078  113.077935   
1           no   1/1/2015 7:00          1  162.879223  402.747490   95.460525   
2           no   1/1/2015 8:00          1  170.989902  527.349825   75.237905   
3           no   1/1/2015 9:00          1  162.462833  346.149335  109.248561   
4           no  1/1/2015 10:00          1  157.610021  435.376873  111.886648   
...        ...             ...        ...         ...         ...         ...   
876540      no   1/1/2016 2:00        100  179.438162  395.222827  102.290715   
876541      no   1/1/2016 3:00        100  189.617555  446.207972   98.180607   
876542      no   1/1/2016 4:00        100  192.483414  447.816524   94.132837   
876543      no   1/1/2016 5:00        100  165.475310  413.771670  104.081073   
876544      no   1/1/2016 6:00        100  171.336037  496.096870   79.095538   

        vibration  volt_lag_1  vibration_lag_1  volt_lag_24  ...  ham_comp1  \
0       45.087686         NaN              NaN          NaN  ...          1   
1       43.413973  176.217853        45.087686          NaN  ...          2   
2       34.178847  162.879223        43.413973          NaN  ...          3   
3       41.122144  170.989902        34.178847          NaN  ...          4   
4       25.990511  162.462833        41.122144          NaN  ...          5   
...           ...         ...              ...          ...  ...        ...   
876540  50.771941  162.742669        44.382754   148.126516  ...       1989   
876541  35.123072  179.438162        50.771941   136.363542  ...       1990   
876542  48.314561  189.617555        35.123072   169.925104  ...       1991   
876543  44.835259  192.483414        48.314561   163.234778  ...       1992   
876544  37.845245  165.475310        44.835259   173.774481  ...       1993   

        ham_comp2  ham_comp3  ham_comp4  pm_comp1  pm_comp2  pm_comp3  \
0               1          1          1         0         0         0   
1               2          2          2         0         0         0   
2               3          3          3         0         0         0   
3               4          4          4         0         0         0   
4               5          5          5         0         0         0   
...           ...        ...        ...       ...       ...       ...   
876540        188       1988       1268         0         0         0   
876541        189       1989       1269         0         0         0   
876542        190       1990       1270         0         0         0   
876543        191       1991       1271         0         0         0   
876544        192       1992       1272         0         0         0   

        pm_comp4   model  age  
0              0  model3   18  
1              0  model3   18  
2              0  model3   18  
3              0  model3   18  
4              0  model3   18  
...          ...     ...  ...  
876540         0  model4    5  
876541         0  model4    5  
876542         0  model4    5  
876543         0  model4    5  
876544         0  model4    5  

[876545 rows x 44 columns]

In [6]:
df.info()
#categorical: failure and model, rest of all are numeric

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876545 entries, 0 to 876544
Data columns (total 44 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   failure                    876545 non-null  object 
 1   datetime                   876545 non-null  object 
 2   machineID                  876545 non-null  int64  
 3   volt                       876545 non-null  float64
 4   rotate                     876545 non-null  float64
 5   pressure                   876545 non-null  float64
 6   vibration                  876545 non-null  float64
 7   volt_lag_1                 876544 non-null  float64
 8   vibration_lag_1            876544 non-null  float64
 9   volt_lag_24                876521 non-null  float64
 10  vibration_lag_24           876521 non-null  float64
 11  volt_rolling_3h            876543 non-null  float64
 12  volt_rolling_min_3h        876543 non-null  float64
 13  volt_rolling_max_3h        87

In [7]:
# Drop the first 24 rows
df = df.drop(df.index[:24])

# Reset the index to have consecutive integer values after removing the rows
df.reset_index(drop=True, inplace=True)

In [8]:
#no missing value confirmation
print(df.isna().sum())

failure                      0
datetime                     0
machineID                    0
volt                         0
rotate                       0
pressure                     0
vibration                    0
volt_lag_1                   0
vibration_lag_1              0
volt_lag_24                  0
vibration_lag_24             0
volt_rolling_3h              0
volt_rolling_min_3h          0
volt_rolling_max_3h          0
volt_rolling_24h             0
volt_rolling_min_24h         0
volt_rolling_max_24h         0
rotate_rolling_min_3h        0
rotate_rolling_max_3h        0
rotate_rolling_24h           0
rotate_rolling_min_24h       0
rotate_rolling_max_24h       0
pressure_rolling_3h          0
pressure_rolling_min_3h      0
pressure_rolling_max_3h      0
pressure_rolling_24h         0
pressure_rolling_min_24h     0
pressure_rolling_max_24h     0
vibration_rolling_3h         0
vibration_rolling_min_3h     0
vibration_rolling_max_3h     0
vibration_rolling_24h        0
vibratio

In [9]:
#Drop unnecessary columns
columns_to_drop = ['datetime', 'machineID', 'pm_comp1', 'pm_comp2', 'pm_comp3', 'pm_comp4']
df1 = df.drop(columns=columns_to_drop)

In [10]:
#getting index location of categoricals
print(df1.columns.get_loc('model'))

36


In [11]:
#encoding categorical features
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Specify the column indices to encode
column_indices = [36]

#use label encoder and override
df1.iloc[:, column_indices]= df1.iloc[:, column_indices].apply(encoder.fit_transform) #direct override, no need to concatenate


<ipython-input-11-d0388fe18ee4>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df1.iloc[:, column_indices]= df1.iloc[:, column_indices].apply(encoder.fit_transform) #direct override, no need to concatenate


In [12]:
#encoding categorical labels
df1['y_var'] = encoder.fit_transform(df['failure'])

In [13]:
df1.y_var.value_counts() #imbalanced

4    875760
1       259
0       192
3       179
2       131
Name: y_var, dtype: int64

In [14]:
#10% random sampling

# Criteria 1: Retain all observations with labels 0, 1, 2, and 3
df2 = df1[df1['y_var'].isin([0, 1, 2, 3])]

# Criteria 2: Sample only 10% of observations with label 4
label_4_df = df1[df1['y_var'] == 4]
sampled_label_4_df = label_4_df.sample(frac=0.1, random_state=42)

# Concatenate both DataFrames to get the final df2
df2 = pd.concat([df2, sampled_label_4_df])

In [15]:
#dependent & independent
y = df2.iloc[:, -1].values #values makes it array
X = df2.iloc[:, 9:38].values

**Solving the problem of imbalanced dataset**

In [16]:
#Balancing using SMOTE
from imblearn.over_sampling import SMOTE

#Apply SMOTE to balance the classes on the whole dataset
smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X, y)

In [17]:
#Results after SMOTE
# Original count of labels in 'y'
original_label_counts = np.bincount(y)
print("Original Label Counts:")
print(original_label_counts)

# Resampled count of labels using SMOTE
resampled_smote_label_counts = np.bincount(y_resampled_smote)
print("\nResampled Label Counts using SMOTE:")
print(resampled_smote_label_counts)

Original Label Counts:
[  192   259   131   179 87576]

Resampled Label Counts using SMOTE:
[87576 87576 87576 87576 87576]


In [18]:
#Split the resampled dataset into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled_smote, y_resampled_smote, test_size=0.2, random_state=42)

In [19]:
#Normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Apply the scaler to transform the training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [20]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test_scaled)

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy
accuracy1 = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy1)

# Calculate precision, recall, and F1-score for each class
precision1 = precision_score(y_test, y_pred, average=None)
recall1 = recall_score(y_test, y_pred, average=None)
f1_score1 = f1_score(y_test, y_pred, average=None)

print("Precision of RF:", precision1)
print("Recall of RF:", recall1)
print("F1-score of RF:", f1_score1)

Accuracy: 0.9953982826345117
Precision of RF: [0.99581182 0.99232649 0.99630724 0.99349253 0.99913189]
Recall of RF: [0.99896403 0.99668874 0.99897465 0.99710359 0.98527565]
F1-score of RF: [0.99738543 0.99450283 0.99763916 0.99529478 0.9921554 ]


In [22]:
#confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix using Seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [23]:
import xgboost as xgb

# Create an XGBoost classifier
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train_scaled, y_train)
y_pred = xgb_classifier.predict(X_test_scaled)

In [ ]:
# Calculate accuracy
accuracy2 = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1-score for each class
precision2 = precision_score(y_test, y_pred, average=None)
recall2 = recall_score(y_test, y_pred, average=None)
f1_score2 = f1_score(y_test, y_pred, average=None)

print("Precision:", precision2)
print("Recall:", recall2)
print("F1-score:", f1_score2)

Accuracy: 0.9956723303188088
Precision: [0.99764395 0.99577674 0.99715796 0.99411365 0.99930999]
Recall: [0.99919424 0.99611784 0.99931643 0.99750114 0.99183883]
F1-score: [0.99841849 0.99594726 0.99823603 0.99580451 0.99556039]


In [25]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Convert target labels to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes=5)
y_test_encoded = to_categorical(y_test, num_classes=5)  # Assuming you have y_test

# Create a sequential neural network model
model = Sequential()

# Add input layer and hidden layers
model.add(Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))

# Add output layer with appropriate activation function (softmax) for multi-class classification
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train_encoded, epochs=16, batch_size=32, validation_split=0.2)

# Make predictions on the test data using the trained neural network
y_pred_probabilities = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probabilities, axis=1)

Epoch 1/16
8758/8758 [==============================] - 42s 4ms/step - loss: 0.1243 - accuracy: 0.9660 - val_loss: 0.0894 - val_accuracy: 0.9769
Epoch 2/16
8758/8758 [==============================] - 37s 4ms/step - loss: 0.0834 - accuracy: 0.9785 - val_loss: 0.0806 - val_accuracy: 0.9787
Epoch 3/16
8758/8758 [==============================] - 35s 4ms/step - loss: 0.0751 - accuracy: 0.9803 - val_loss: 0.0747 - val_accuracy: 0.9797
Epoch 4/16
8758/8758 [==============================] - 35s 4ms/step - loss: 0.0701 - accuracy: 0.9815 - val_loss: 0.0688 - val_accuracy: 0.9825
Epoch 5/16
8758/8758 [==============================] - 35s 4ms/step - loss: 0.0662 - accuracy: 0.9824 - val_loss: 0.0667 - val_accuracy: 0.9822
Epoch 6/16
8758/8758 [==============================] - 35s 4ms/step - loss: 0.0629 - accuracy: 0.9833 - val_loss: 0.0681 - val_accuracy: 0.9825
Epoch 7/16
8758/8758 [==============================] - 36s 4ms/step - loss: 0.0608 - accuracy: 0.9835 - val_loss: 0.0620 - val_ac

In [24]:
y_pred_probabilities = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probabilities, axis=1)

NameError: ignored

In [ ]:
#ANN performance metrics
# Calculate accuracy
accuracy3 = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1-score for each class
precision3 = precision_score(y_test, y_pred, average=None)
recall3 = recall_score(y_test, y_pred, average=None)
f1_score3 = f1_score(y_test, y_pred, average=None)

print("Precision:", precision3)
print("Recall:", recall3)
print("F1-score:", f1_score3)

Accuracy: 0.9956723303188088
Precision: [0.98130257 0.97872819 0.98515102 0.98648343 1.        ]
Recall: [0.99378417 0.99292076 0.99772145 0.99062926 0.95588403]
F1-score: [0.98750393 0.98577339 0.99139639 0.988552   0.97744449]
